# COG GDAL ENV Var Tests

## Explanation

In this notebook we demonstrate the importance of using GDAL ENV variables.

GDAL environment variables are an important factor to delivering performant tiling using rasterio.

These GDAL variables are documented here https://developmentseed.org/titiler/advanced/performance_tuning/. Those comments are also in cog_tile_test.py.

We run 3 iterations of generating tiles from zoom 0 to 12 for tiling when GDAL environment variables are set and when they are unset and display the results.

## Setup

In [ ]:
import json
import pandas as pd
from cog_tile_test import CogTileTest

import sys; sys.path.append('..')
import helpers.eodc_hub_role as eodc_hub_role

In [ ]:
credentials = eodc_hub_role.fetch_and_set_credentials()

In [ ]:
# Run 3 iterations of each setting
iterations = 3
zooms = range(12)
dataset_id, dataset = json.loads(open('../01-generate-datasets/cog-datasets.json').read()).items()


## Run tests

In [ ]:

# Create a test with gdal vars unset
cog_tile_test_unset = CogTileTest(
    dataset_id=dataset_id,
    extra_args={
        'query': dataset['example_query'],
        'set_gdal_vars': False
    }
)

# Run it 3 times for each zoom level
for zoom in zooms:
    cog_tile_test_unset.run_batch({'zoom': zoom}, batch_size=iterations)

unset_results = cog_tile_test_unset.store_results(credentials)


In [ ]:
# Create a test with gdal vars SET
cog_tile_test_set = CogTileTest(
    dataset_id=dataset_id,
    extra_args={
        'query': dataset['example_query'],
        'set_gdal_vars': True
    }
)

# Run it 3 times for each zoom level
for zoom in zooms:
    cog_tile_test_set.run_batch({'zoom': zoom}, batch_size=iterations)

set_results = cog_tile_test_set.store_results(credentials)

# Read + Plot results 

In [ ]:
# see code in run-xarray-tests.ipynb

In [ ]:
expanded_df.plot.scatter(x='zoom', y='time', by='gdal_vars_set')

In [ ]:
expanded_df.results.to_csv('results/cog-gdal-results.csv')